In [1]:
import os
import time
import json
import cv2
import numpy as np
import torch
from torch import nn
import pickle
from torch.autograd import Variable
from matplotlib import pyplot as plt
import torch.nn.functional as F

In [2]:
from cnn import SegmentationModel as net

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
def calc_b(total_dim, input_dim, margin):
    n = int(np.ceil(total_dim/input_dim))
    if n == 1:
        xo = x = np.array([[0], [total_dim]])
        return(x, xo)
    
    out_dim = total_dim/n

    c = np.repeat(np.array([[-1, 1]]), n, 0)
    c[[0,-1]] = np.array([[0,1], [-1,0]])
    c = np.int0(margin * c.transpose())

    b = out_dim * (0.5 + np.arange(n))
    b0 = b - 0.5 * out_dim
    b1 = b0 + out_dim
    bb = np.stack([b0, b1])

    x = np.int0(c + bb)

    xdim = np.int0(16 * np.round((x[1] - x[0])/16))
    x[0,1:] = x[1,1:] - xdim[1:]
    x[1,0] = x[0,0] + xdim[0]
    xo = np.int0(bb - x)
    xo[1] += x[1] - x[0]
    return(x, xo)

def calc_bb(tw, th, iw, ih):
    xc, xco = calc_b(tw, iw, 160)
    yc, yco = calc_b(th, ih, 160)
    return([xc, yc, xco, yco])

In [5]:
n_classes = 2

# Atribui o modelo
model = net.EESPNet_Seg(n_classes, s=1.5, pretrained='', gpus=1)
model = model.to(device)
model.load_state_dict(torch.load('../data/model_seg_LR_bestt.pth',map_location=device))


# set to evaluation mode
model.eval()

print('Model OK')

Model OK


In [6]:
im_dir = '/home/rodrigo7/Documents/Milho_HTPP/Recortes/'
s_dir = '/home/rodrigo7/Documents/Milho_HTPP/Recortes/'
img_ext = '.jpg'
ims = sorted(os.listdir(im_dir))
ims = [i for i in ims if i.endswith(img_ext) and 'Dia' in i]
ims


['Gleba_G1a_Plot_0_Dia_23.jpg',
 'Gleba_G1a_Plot_0_Dia_27.jpg',
 'Gleba_G1a_Plot_0_Dia_30.jpg',
 'Gleba_G1a_Plot_0_Dia_33.jpg',
 'Gleba_G1a_Plot_100_Dia_23.jpg',
 'Gleba_G1a_Plot_100_Dia_27.jpg',
 'Gleba_G1a_Plot_100_Dia_30.jpg',
 'Gleba_G1a_Plot_100_Dia_33.jpg',
 'Gleba_G1a_Plot_101_Dia_23.jpg',
 'Gleba_G1a_Plot_101_Dia_27.jpg',
 'Gleba_G1a_Plot_101_Dia_30.jpg',
 'Gleba_G1a_Plot_101_Dia_33.jpg',
 'Gleba_G1a_Plot_102_Dia_23.jpg',
 'Gleba_G1a_Plot_102_Dia_27.jpg',
 'Gleba_G1a_Plot_102_Dia_30.jpg',
 'Gleba_G1a_Plot_102_Dia_33.jpg',
 'Gleba_G1a_Plot_103_Dia_23.jpg',
 'Gleba_G1a_Plot_103_Dia_27.jpg',
 'Gleba_G1a_Plot_103_Dia_30.jpg',
 'Gleba_G1a_Plot_103_Dia_33.jpg',
 'Gleba_G1a_Plot_104_Dia_23.jpg',
 'Gleba_G1a_Plot_104_Dia_27.jpg',
 'Gleba_G1a_Plot_104_Dia_30.jpg',
 'Gleba_G1a_Plot_104_Dia_33.jpg',
 'Gleba_G1a_Plot_105_Dia_23.jpg',
 'Gleba_G1a_Plot_105_Dia_27.jpg',
 'Gleba_G1a_Plot_105_Dia_30.jpg',
 'Gleba_G1a_Plot_105_Dia_33.jpg',
 'Gleba_G1a_Plot_106_Dia_23.jpg',
 'Gleba_G1a_Plot_106_D

In [7]:
def transform(x):
    x = x.copy()
    x = x.astype('float')
    x -= 128
    x /= 35
    return(x)

In [8]:
#Define o mapa de cores para geração da máscara
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 1] = [255,0,0]
    msk[cat == 2] = [0,0,255]
    return(msk)

In [9]:
def get_pts(pred, c = 1):
    msk = (pred[[c]])
    mskl = F.max_pool2d(msk, stride=1, kernel_size=25, padding=12)  
    crop =  pred.argmax(0) == c
    pts = np.stack(np.where(((mskl == msk) * crop).cpu().data.numpy()))[[2, 1, 0]].transpose()
    pts[:,2] = c
    return (np.int0(pts))

In [10]:
def coords_to_json(coords, filename):    
    regions = []
    for cat_name in coords.keys():
        region_attri = {"Planta":cat_name}
        for pt in coords.get(cat_name):
            pt = {"name":"point","cx":int(pt[0]),"cy":int(pt[1])} 
            regions.append({'region_attributes':region_attri, 'shape_attributes':pt})

    file_size = int(os.path.getsize(filename))
    img_name = os.path.basename(filename)
    file_attri = {'filename': img_name,
                  'size' : file_size,
                  'regions' : regions,
                  'file_attributes': {}}
    via_img_metadata = {img_name + str(file_size):file_attri}
    return(via_img_metadata)

In [11]:
all_json = {}
time0 = time.time()
for i in ims:
    start_time = time.time()
    print(i)
    img_name = os.path.join(im_dir, i)
    new_img_name =  os.path.join(s_dir, i[:-4] + '_pred.png')
#     if os.path.exists(new_img_name):
#         pass
# #         continue

    imgo = cv2.imread(img_name)
    img = transform(imgo)
    
    input_height = 2880
    input_width = 2880
    
    total_width, total_height = img.shape[:2]
    xc, yc, xco, yco = calc_bb(total_width, total_height, input_height, input_width)

    time_taken = time.time() - start_time
    print('PreProc time: %.2f' % time_taken)

    im = np.moveaxis(img, 2, 0)
    img_tensor = torch.from_numpy(im)
    img_variable = Variable(img_tensor.unsqueeze(0)).to(device, dtype=torch.float)
    with torch.no_grad():
        pred, feat = model(img_variable)
    
    time_taken = time.time() - start_time - time_taken
    print('Prediction time: %.2f' % time_taken)
    
    pred = pred[0]
    cat = pred.argmax(0)
    predm = F.max_pool2d(pred[[1]], stride=1, kernel_size=25, padding=12)  
    pts = np.stack(np.where(((predm == pred[[1]]) * (cat == 1)).cpu().data.numpy()))[[2, 1, 0]].transpose()

    coords_json = {'Pendao':pts}
    coords_json = coords_to_json(coords_json, img_name)
    all_json.update(coords_json)
    
    pts[:,1] *= -1
    pts_name =  os.path.join(s_dir, i[:-4] + '_pred.csv')
    np.savetxt(pts_name, pts, delimiter= ',', header='lon,lat,class', comments='')
    print(len(pts))
    
    cat = cat.cpu().data.numpy()
    pred_col = colorir(cat)
    cv2.imwrite(new_img_name, pred_col)
   
    time_taken = time.time() - start_time
    print('Total time: %.2f' % time_taken)
    
time_taken = time.time() - time0
print('Final time: %.2f' % time_taken)





Gleba_G1a_Plot_0_Dia_23.jpg
PreProc time: 0.03
Prediction time: 0.62
0
Total time: 0.68
Gleba_G1a_Plot_0_Dia_27.jpg
PreProc time: 0.02
Prediction time: 0.02
56
Total time: 0.05
Gleba_G1a_Plot_0_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.05
Gleba_G1a_Plot_0_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
45
Total time: 0.04
Gleba_G1a_Plot_100_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_100_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
70
Total time: 0.05
Gleba_G1a_Plot_100_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
64
Total time: 0.04
Gleba_G1a_Plot_100_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
62
Total time: 0.05
Gleba_G1a_Plot_101_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_101_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
56
Total time: 0.04
Gleba_G1a_Plot_101_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
62
Total time: 0.04
Gleba_G1a_

Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_120_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
68
Total time: 0.05
Gleba_G1a_Plot_120_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
65
Total time: 0.05
Gleba_G1a_Plot_120_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
58
Total time: 0.05
Gleba_G1a_Plot_121_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_121_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
71
Total time: 0.05
Gleba_G1a_Plot_121_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
56
Total time: 0.05
Gleba_G1a_Plot_121_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
56
Total time: 0.04
Gleba_G1a_Plot_122_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_122_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
68
Total time: 0.06
Gleba_G1a_Plot_122_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
64
Total time: 0.05
Gleba_G1a_Plot_122_Dia_33.jpg
PreProc time: 0.01
Pr

PreProc time: 0.01
Prediction time: 0.02
53
Total time: 0.05
Gleba_G1a_Plot_142_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_142_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
62
Total time: 0.04
Gleba_G1a_Plot_142_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.03
58
Total time: 0.05
Gleba_G1a_Plot_142_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
69
Total time: 0.05
Gleba_G1a_Plot_143_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_143_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.04
Gleba_G1a_Plot_143_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
51
Total time: 0.04
Gleba_G1a_Plot_143_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
58
Total time: 0.04
Gleba_G1a_Plot_144_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_144_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
64
Total time: 0.04
Gleba_G1a_Plot_144_Dia_30.jpg
Pr

Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_163_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.04
Gleba_G1a_Plot_163_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
53
Total time: 0.04
Gleba_G1a_Plot_163_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
55
Total time: 0.04
Gleba_G1a_Plot_164_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_164_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
51
Total time: 0.04
Gleba_G1a_Plot_164_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
55
Total time: 0.04
Gleba_G1a_Plot_164_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
55
Total time: 0.04
Gleba_G1a_Plot_165_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_165_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
59
Total time: 0.04
Gleba_G1a_Plot_165_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
59
Total time: 0.05
Gleba_G1a_Plot_165_Dia_33.jpg
PreProc time: 0.01
Pr

Prediction time: 0.02
59
Total time: 0.05
Gleba_G1a_Plot_184_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
60
Total time: 0.05
Gleba_G1a_Plot_184_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
62
Total time: 0.04
Gleba_G1a_Plot_185_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_185_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.04
Gleba_G1a_Plot_185_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.05
Gleba_G1a_Plot_185_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
60
Total time: 0.04
Gleba_G1a_Plot_186_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G1a_Plot_186_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
62
Total time: 0.05
Gleba_G1a_Plot_186_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
59
Total time: 0.05
Gleba_G1a_Plot_186_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
58
Total time: 0.04
Gleba_G1a_Plot_187_Dia_23.jpg
PreProc time: 0.01
P

...

Prediction time: 0.02
38
Total time: 0.05
Gleba_G5b_Plot_24_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
45
Total time: 0.04
Gleba_G5b_Plot_250_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_250_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
59
Total time: 0.04
Gleba_G5b_Plot_250_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
21
Total time: 0.04
Gleba_G5b_Plot_250_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
11
Total time: 0.04
Gleba_G5b_Plot_251_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
12
Total time: 0.04
Gleba_G5b_Plot_251_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
49
Total time: 0.05
Gleba_G5b_Plot_251_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
47
Total time: 0.05
Gleba_G5b_Plot_251_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
35
Total time: 0.04
Gleba_G5b_Plot_252_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_252_Dia_27.jpg
PreProc time: 0.01
Pr

Prediction time: 0.02
64
Total time: 0.05
Gleba_G5b_Plot_270_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
49
Total time: 0.05
Gleba_G5b_Plot_270_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
44
Total time: 0.04
Gleba_G5b_Plot_271_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_271_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
70
Total time: 0.05
Gleba_G5b_Plot_271_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
42
Total time: 0.05
Gleba_G5b_Plot_271_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
44
Total time: 0.04
Gleba_G5b_Plot_272_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
8
Total time: 0.04
Gleba_G5b_Plot_272_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
62
Total time: 0.04
Gleba_G5b_Plot_272_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
42
Total time: 0.05
Gleba_G5b_Plot_272_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
49
Total time: 0.05
Gleba_G5b_Plot_273_Dia_23.jpg
PreProc time: 0.01
P

0
Total time: 0.04
Gleba_G5b_Plot_39_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
68
Total time: 0.04
Gleba_G5b_Plot_39_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
39
Total time: 0.04
Gleba_G5b_Plot_39_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
43
Total time: 0.04
Gleba_G5b_Plot_3_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
34
Total time: 0.04
Gleba_G5b_Plot_3_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
65
Total time: 0.05
Gleba_G5b_Plot_3_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.05
Gleba_G5b_Plot_3_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
58
Total time: 0.05
Gleba_G5b_Plot_40_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
1
Total time: 0.04
Gleba_G5b_Plot_40_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
69
Total time: 0.04
Gleba_G5b_Plot_40_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
53
Total time: 0.05
Gleba_G5b_Plot_40_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
47
Total time: 0

Prediction time: 0.02
36
Total time: 0.05
Gleba_G5b_Plot_5_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
47
Total time: 0.04
Gleba_G5b_Plot_60_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_60_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
81
Total time: 0.05
Gleba_G5b_Plot_60_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
14
Total time: 0.04
Gleba_G5b_Plot_60_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
38
Total time: 0.04
Gleba_G5b_Plot_61_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_61_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
63
Total time: 0.04
Gleba_G5b_Plot_61_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
46
Total time: 0.05
Gleba_G5b_Plot_61_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
54
Total time: 0.05
Gleba_G5b_Plot_62_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
15
Total time: 0.04
Gleba_G5b_Plot_62_Dia_27.jpg
PreProc time: 0.01
Prediction ti

PreProc time: 0.01
Prediction time: 0.02
49
Total time: 0.04
Gleba_G5b_Plot_81_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_81_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
74
Total time: 0.05
Gleba_G5b_Plot_81_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
47
Total time: 0.05
Gleba_G5b_Plot_81_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
47
Total time: 0.05
Gleba_G5b_Plot_82_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_82_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
69
Total time: 0.04
Gleba_G5b_Plot_82_Dia_30.jpg
PreProc time: 0.01
Prediction time: 0.02
49
Total time: 0.04
Gleba_G5b_Plot_82_Dia_33.jpg
PreProc time: 0.01
Prediction time: 0.02
36
Total time: 0.04
Gleba_G5b_Plot_83_Dia_23.jpg
PreProc time: 0.01
Prediction time: 0.02
0
Total time: 0.04
Gleba_G5b_Plot_83_Dia_27.jpg
PreProc time: 0.01
Prediction time: 0.02
82
Total time: 0.05
Gleba_G5b_Plot_83_Dia_30.jpg
PreProc time:

In [12]:
with open('../data/pred_via_region_data_final.json', 'w') as jsonfile:
    json.dump(all_json, jsonfile)